<a href="https://colab.research.google.com/github/lapatradaa/M-MMT4NL/blob/main/llms_evaluation_direct_translate_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install sacrebleu bert-score
!pip install sacrebleu bert-score pythainlp
!pip -q install pandas openpyxl jellyfish bert-score tqdm openai --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 116.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 138.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.9/355.9 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.4/948.4 kB 61.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.2 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.2 which is incompat

In [4]:
import os, random, math
from pathlib import Path
import pandas as pd
import random
import os
from tqdm import tqdm
import jellyfish
from bert_score import score as bertscore


In [5]:
OPENAI_MODEL_TRANSLATE = "gpt-4o-mini"
OPENAI_MODEL_PERTURB  = "gpt-4o-mini"
TEMPERATURE_TRANSLATE  = 0.2
TEMPERATURE_PERTURB    = 0.7


In [6]:
# multilingual BERTScore model (works well for Thai)
BERT_MODEL = "xlm-roberta-large"
W_BERT = 0.8
W_JARO = 0.2

RANDOM_SEED = 42
random.seed(RANDOM_SEED)

In [7]:
from openai import OpenAI

##_client = OpenAI <“OPENAI_API_KEY”>

def _chat_once(system_prompt: str, user_prompt: str, *, model: str, temperature: float) -> str:
    out = _client.chat.completions.create(
        model=model,
        temperature=temperature,
        messages=[
            {"role": "system", "content": system_prompt.strip()},
            {"role": "user",   "content": user_prompt.strip()},
        ],
    )
    return (out.choices[0].message.content or "").strip()

In [8]:
# --- Prompt Templates ---
PROMPT_TAXONOMY = """You're an expert linguist in English and Thai. You need to modify this Thai sentence by substituting a word with its respective synonym, while still keeping the whole semantic of the sentence.

Examples of this modifications in English are as follows.

I'll give you some examples of converting one sentence to another sentence: "I'm so tired" is converted to "I'm so exhausted" "I'm really hungry" is converted to "I'm really starving" "I'm not sure if I'm up for that" is converted to "I'm not certain if I'm up for that" "I'm not sure if I can make it to the event" is converted to "I'm not confident if I can make it to the event"

Can you apply this concept to the Thai sentence below. Only show the modified sentence without any explanation.
You're an expert linguist in English and Thai. You need to modify this Thai sentence by negating the sentence, while still keeping the whole semantic of the sentence.

Examples of this modifications in English are as follows.

"I'm so tired" is converted to "I'm so not energetic"
"I'm really hungry" is converted to "I'm really not full"
"I'm not sure if I'm up for that" is converted to "I'm sure I'm not up for that"
"I'm not sure if I can make it to the event" is converted to "I'm unsure if I can make it to the event"
"I'm feeling a bit confused right now" is converted to "I'm feeling a bit not clear right now"

Can you apply this concept to the Thai sentence below. Only show the modified sentence without any explanation.

{thai_text}

"""


In [9]:
PROMPT_NER = """Can you apply this concept to the Thai sentence below. Only show the modified sentence without any explanation.
You're an expert linguist in English and Thai. You need to modify this Thai sentence by replacing the named entity with a new name, while still keeping the whole semantic of the sentence.

Examples of this modifications in English are as follows.

"I'm so tired" is converted to "Jane is so tired"
"I'm really hungry" is converted to "Jack is really hungry" "I'm not sure if I'm up for that" is converted to "Jones is not sure if she is up for that"
"I'm not sure if I can make it to the event" is converted to "Jill is not sure if she can make it to the event"
"I'm feeling a bit confused right now" is converted to "Andy is feeling a bit confused right now"

Can you apply this concept to the Thai sentence below. Only show the modified sentence without any explanation.

{thai_text}
"""

In [10]:
def perturb_thai(thai_text: str, taxonomy: str):
    if taxonomy == "synonym":
        prompt = PROMPT_SYNONYM.format(thai_text=thai_text)
    elif taxonomy == "negation":
        prompt = PROMPT_NEGATION.format(thai_text=thai_text)
    elif taxonomy == "ner":
        prompt = PROMPT_NER.format(thai_text=thai_text)
    else:
        raise ValueError("taxonomy must be synonym / negation / ner")

    resp = _client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role":"system","content":"You are a helpful assistant."},
                  {"role":"user","content":prompt}],
        temperature=0.7
    )
    return resp.choices[0].message.content.strip()

In [11]:
def jaro(a: str, b: str) -> float:
    try:
        return float(jellyfish.jaro_similarity(a or "", b or ""))
    except Exception:
        return 0.0

In [12]:
def bert_f1(ref: str, cand: str) -> float:
    if not ref.strip() or not cand.strip():
        return 0.0
    try:
        P, R, F1 = bertscore(
            cands=[cand],
            refs=[ref],
            model_type="xlm-roberta-large",
            lang="th",  # ภาษาไทย
            rescale_with_baseline=False,  # non-baseline for th
            verbose=False
        )
        return float(F1[0].item())
    except Exception:
        return 0.0

In [13]:
def fitness(ref_thai: str, cand_thai: str, W_BERT=0.8, W_JARO=0.2):
    b = bert_f1(ref_thai, cand_thai)
    j = jaro(ref_thai, cand_thai)
    f = W_BERT * b + W_JARO * j
    return round(f, 4), round(b, 4), round(j, 4)

In [23]:
# Evaluate Sentence with Perturbation
# --------------------------
def evaluate_sentence(sentence: str, taxonomy: str, rounds: int = 3, perturb_type: str = "taxonomy", threshold: float = 0.9):
    """
    Evaluate model robustness under perturbations.

    Args:
        sentence (str): Original Thai sentence.
        taxonomy (str): Perturbation taxonomy type (e.g. "Negation").
        rounds (int): Number of perturbation rounds per sentence.
        perturb_type (str): Either "taxonomy" or "ner".
        threshold (float): BERTScore threshold for pass/fail.

    Returns:
        list of dict: Each dict contains original, perturbed, responses, BERTScore, and pass/fail.
    """
    results = []
    for _ in range(rounds):

        # --- Select correct perturbation prompt ---
        if perturb_type == "taxonomy":
            prompt = PROMPT_TAXONOMY.format(thai_text=sentence)
        elif perturb_type == "ner":
            prompt = PROMPT_NER.format(thai_text=sentence)
        else:
            raise ValueError("Invalid perturb_type. Choose 'taxonomy' or 'ner'.")

        # --- Generate perturbed sentence ---
        perturbed = _chat_once(
            system_prompt="You are a helpful Thai linguist.",
            user_prompt=prompt,
            model="gpt-4o-mini",
            temperature=0.7
        )

        # --- Evaluate original ---
        model_output_original = _chat_once(
            system_prompt="You are a helpful assistant.",
            user_prompt=sentence,
            model="gpt-4o-mini",
            temperature=0
        )

        # --- Evaluate perturbed ---
        model_output_perturbed = _chat_once(
            system_prompt="You are a helpful assistant.",
            user_prompt=perturbed,
            model="gpt-4o-mini",
            temperature=0
        )

        # --- Compute BERTScore similarity ---
        similarity = bert_f1(model_output_original, model_output_perturbed)
        passed = similarity >= threshold

        # --- Save results ---
        results.append({
            "taxonomy": taxonomy,
            "original": sentence,
            "perturb_type": perturb_type,
            "perturbed": perturbed,
            "original_response": model_output_original,
            "perturbed_response": model_output_perturbed,
            "bert_f1": similarity,
            "pass": passed
        })

    return results


In [24]:
def run_file(infile: str, outfile: str, taxonomy: str, perturb_type: str, rounds: int = 3):
    df = pd.read_excel(infile)
    if "original" not in df.columns:
        raise ValueError("Excel file ต้องมี column 'original' ที่เป็นประโยคภาษาไทย")

    results = []
    for _, row in tqdm(df.iterrows(), total=len(df), desc=f"Evaluating {perturb_type}"):
        thai_text = str(row["original"]).strip()
        if not thai_text:
            continue
        res = evaluate_sentence(thai_text, taxonomy, rounds=rounds, perturb_type=perturb_type)
        results.extend(res)

    out_df = pd.DataFrame(results)
    out_df.to_excel(outfile, index=False)
    print(f"✅ {perturb_type} results saved to {outfile}")
    return out_df


In [25]:
# Run separately
run_file("/content/Test2_mmmt4nl (1).xlsx", "output_taxonomy.xlsx", taxonomy="Negation", perturb_type="taxonomy", rounds=3)
run_file("/content/Test2_mmmt4nl (1).xlsx", "output_ner.xlsx", taxonomy="Negation", perturb_type="ner", rounds=3)

Evaluating taxonomy:   0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

...

Evaluating taxonomy: 100%|██████████| 10/10 [05:18<00:00, 31.90s/it]


✅ taxonomy results saved to output_taxonomy.xlsx


Evaluating ner: 100%|██████████| 10/10 [04:41<00:00, 28.14s/it]

✅ ner results saved to output_ner.xlsx


,taxonomy,original,perturb_type,perturbed,original_response,perturbed_response,bert_f1,pass
0,Negation,ฉันมีความสุขมากที่ได้พบเพื่อนใหม่,ner,ฉันมีความสุขมากที่ได้พบสมชายใหม่,ดีใจที่คุณมีความสุขนะ! การได้พบเพื่อนใหม่เป็นส...,ดีใจที่คุณมีความสุขนะคะ! การพบคนใหม่สามารถนำมา...,0.904237,True
1,Negation,ฉันมีความสุขมากที่ได้พบเพื่อนใหม่,ner,ฉันมีความสุขมากที่ได้พบพี่ใหม่,ดีใจที่ได้ยินว่าคุณมีความสุข! การได้พบเพื่อนให...,ดีใจที่ได้ยินว่าคุณมีความสุข! การได้พบคนใหม่ๆ ...,0.908596,True
2,Negation,ฉันมีความสุขมากที่ได้พบเพื่อนใหม่,ner,ฉันมีความสุขมากที่ได้พบสมชายใหม่,ดีใจที่ได้ยินว่าคุณมีความสุข! การได้พบเพื่อนให...,ดีใจที่คุณมีความสุขนะคะ! สมชายใหม่เป็นคนที่ทำใ...,0.889486,False
3,Negation,วันนี้อากาศดีและสดชื่น,ner,วันนี้อากาศดีและสดชื่นของพิมพ์ชนก,ดีใจที่ได้ยินว่าอากาศดีและสดชื่น! เป็นวันที่เห...,ดีใจที่ได้ยินว่าพิมพ์ชนกมีวันที่อากาศดีและสดชื...,0.927079,True
4,Negation,วันนี้อากาศดีและสดชื่น,ner,วันนี้อากาศดีและสดชื่นของน้องใหม่,ดีใจที่ได้ยินว่าอากาศดีและสดชื่น! วันแบบนี้เหม...,วันนี้อากาศดีและสดชื่นจริง ๆ ค่ะ เป็นวันที่เหม...,0.897342,False
5,Negation,วันนี้อากาศดีและสดชื่น,ner,วันนี้อากาศดีและสดชื่นของมาลี,ดีใจที่ได้ยินว่าอากาศดีและสดชื่น! เป็นวันที่เห...,ดีใจที่ได้ยินว่าคุณมีวันที่อากาศดีและสดชื่นในม...,0.929814,True
6,Negation,หล่อนรู้สึกเสียใจที่ไม่สามารถไปงานได้,ner,นางสาวปรียานุชรู้สึกเสียใจที่ไม่สามารถไปงานได้,หล่อนรู้สึกเสียใจที่ไม่สามารถไปงานได้ เพราะเธอ...,นางสาวปรียานุชอาจรู้สึกเสียใจที่ไม่สามารถไปงาน...,0.888570,False
7,Negation,หล่อนรู้สึกเสียใจที่ไม่สามารถไปงานได้,ner,นางสาวปารีสรู้สึกเสียใจที่ไม่สามารถไปงานได้,หล่อนรู้สึกเสียใจที่ไม่สามารถไปงานได้ เพราะเธอ...,นางสาวปารีสอาจรู้สึกเสียใจเพราะงานนั้นมีความสำ...,0.911245,True
8,Negation,หล่อนรู้สึกเสียใจที่ไม่สามารถไปงานได้,ner,เจนรู้สึกเสียใจที่ไม่สามารถไปงานได้,หล่อนรู้สึกเสียใจที่ไม่สามารถไปงานได้ เพราะเธอ...,เข้าใจความรู้สึกของเจนค่ะ บางครั้งเราก็มีเหตุผ...,0.887176,False
9,Negation,อาหารมื้อนี้อร่อยสุดๆ,ner,อาหารมื้อนี้อร่อยสุดๆ ของสมชาย,ดีใจที่ได้ยินว่าคุณมีอาหารมื้อนี้อร่อยสุดๆ! คุ...,ดีใจที่ได้ยินว่าสมชายมีอาหารมื้อนี้อร่อยสุดๆ! ...,0.941113,True
